# Python PIL (Pillow) benchmark
Since the Julia Benchmark package [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) automatically selects the number of samples, the Python benchmarks (computed using [timeit](https://github.com/python/cpython/blob/3.8/Lib/timeit.py)) are configured according to the number of samples BenchmarkTools takes.

Our tests consist of the following benchmarks:
1. Loading images
2. Saving images
3. Resizing (Scaling) images
4. Rotating Images
5. Getting the histogram (Grayscale Histogram)
6. Equalizing the Grayscale Histogram
7. Converting image to grayscale
8. Changing colorspace to HSV
9. Applying Gaussian Blur

In [3]:
import PIL
import timeit

## 1. Loading images

In [4]:
s = """\
img = Image.open("images/lighthouse.png")
"""

timeit.timeit(stmt = s, setup = "from PIL import Image", number = 44)

0.07964240000001155

## 2. Saving images

In [5]:
s = """\
img.save("imagecopy.png")
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

timeit.timeit(stmt = s, setup = setup, number = 50)

15.602969199999961

## 3. Resizing (Scaling) images

In [6]:
s = """\
xsize, ysize = img.size
img.resize((xsize*3, ysize*3))
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

timeit.timeit(stmt = s, setup = setup, number = 22)

0.426873999999998

## 4. Rotating Images

In [7]:
s = """\
img.rotate(90)
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

timeit.timeit(stmt = s, setup = setup, number = 184)

0.669457500000135

## 5. Getting the histogram (Grayscale Histogram)

In [8]:
s = """\
img.histogram()
"""

setup = """\
from PIL import Image
img = Image.open("images/lighthouse.png")
"""

timeit.timeit(stmt = s, setup = setup, number = 125)

0.2962422000000515

## 6. Equalizing the Grayscale Histogram

In [9]:
s = """\
PIL.ImageOps.equalize(img)
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

timeit.timeit(stmt = s, setup = setup, number = 63)

0.35216560000003483

## 7. Converting image to grayscale

In [10]:
s = """\
PIL.ImageOps.grayscale(img)
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

timeit.timeit(stmt = s, setup = setup, number = 1270)

1.0791987000000063

## 8. Changing colorspace to HSV

In [13]:
s = """\
img.convert('HSV')
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageOps
"""

timeit.timeit(stmt = s, setup = setup, number = 362)

9.150982999999542

## 9. Applying Gaussian Blur

In [12]:

s = """\
gauss = img.filter(PIL.ImageFilter.GaussianBlur(radius=3))
"""

setup = """\
import PIL
img = PIL.Image.open("images/lighthouse.png")
from PIL import ImageFilter
"""

timeit.timeit(stmt = s, setup = setup, number = 75)

2.499535799999876

## Summary:

Summarising, we get the following times:

(Values rounded off to 4 decimal points)

1. Loading images: `0.0796s`
2. Saving images: `15.6030s`
3. Resizing (Scaling) images: `0.4269s`
4. Rotating Images: `0.6695s`
5. Getting the histogram (Grayscale Histogram): `0.2962s`
6. Equalizing the Grayscale Histogram: `0.3522s`
7. Converting image to grayscale: `1.0792s`
8. Changing colorspace to HSV: `9.1510s`
9. Applying Gaussian Blur: `2.4995s`

## Table for comparison

Here is the table of the Benchmarks:

| Testcase/Framework               | Images.jl | Pillow (PIL) | OpenCV   |
|----------------------------------|-----------|--------------|----------|
| Loading images                   | 5.0217s   | 0.9200s      | 0.0796s  |
| Saving images                    | 5.0001s   | 2.0966s      | 15.6030s |
| Resizing (Scaling) images        | 5.0767s   | 0.1671s      | 0.4269s  |
| Rotating images                  | 4.9978s   | 0.7836s      | 0.6695s  |
| Calculating Greyscale Histogram  | 4.9971s   | 0.0594s      | 0.2962s  |
| Greyscale Histogram equalization | 5.0531s   | 0.0032s      | 0.3522s  |
| Converting to Greyscale          | 4.9924s   | 0.2903s      | 1.0792s  |
| Changing colorspace to HSV       | 4.9963s   | 0.6046s      | 9.1510s  |
| Applying Gaussian Blur           | 5.0297s   | 0.0768s      | 2.4995s  |

First off, we can see that BenchmarkTools.jl sets the number of samples such that the benchmarks all take 5 seconds in total. We find that Pillow (PIL) consistently gives sub zero-times for the operations with the one outlier of saving images, while OpenCV varies wildly and takes signficantly more time for some basic tasks such as opening, changing color spaces, converting to grayscale and HSV. 

Comparing Images.jl to the other 2 frameworks, Images.jl is never the fastest, which is a point where significant improvement can be made. However, Images.jl compares well with the two other framework when the operation includes changing the colorspace and saving images. Another point of concern is the fact that Images.jl is the slowest of the 3 frameworks in 7/9 of the tests. This might be due to OpenCV's powerful C and C++ bindings, or because of the powerful and fast `numpy` backend that Python has.